In [1]:
import pandas as pd
from itertools import permutations
import collections

In [2]:
def get_code_to_color(df):
    if len(df.iloc[0]['Genotype'])==6:
        df['gene_series'] = df.Genotype.apply(lambda x: x[:2]+'-'+x[2:4]+'-'+x[4:])
    else:
        df['gene_series'] = df.Genotype.apply(lambda x: x[:2]+'-'+x[2:4]+'-'+x[4:6]+'-'+x[6:])
    return pd.Series(df.Color.values, index= df.gene_series).to_dict()

In [3]:
roses_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Roses'))
cosmos_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Cosmos'))
lilies_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Lilies'))
pansies_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Pansies'))
hyacinths_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Hyacinths'))
tulips_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Tulips'))
mums_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Mums'))
windflowers_dict = get_code_to_color(pd.read_excel('flower_code.xlsx',sheet_name='Windflowers'))

In [4]:
flower_dict = {
    'rose':roses_dict,
    'cosmo':cosmos_dict,
    'lily':lilies_dict,
    'pansy':pansies_dict,
    'hyacinth':hyacinths_dict,
    'tulip':tulips_dict,
    'mum':mums_dict,
    'windflower':windflowers_dict
}

In [5]:
class flowerGenetic(object):
    def __init__(self, flower_name, gene_series):
        self.gene_series = gene_series
        self.flower_name = flower_name
        self.color = self.set_color()
        
    def get_gene_series(self):
        return self.gene_series
    
    def get_color(self):
        return self.color
    
    def set_color(self):
        color_dict = flower_dict[self.flower_name]
        if self.gene_series in color_dict.keys():
            return color_dict[self.gene_series]
        else:
            return 'invalid'
    
    def combine(self, gene1, gene2):
        gene1X = gene1[0]
        gene1Y = gene1[1]
        
        gene2X = gene2[0]
        gene2Y = gene2[1]
        
        genes = list(set([gene1X+gene2X,
                gene1X+gene2Y,
                gene1Y+gene2X,
                gene1Y+gene2Y]))
    
        new_gene = []
        for gene in genes:
            if gene[0] == gene[1]:
                new_gene.append(gene)
            else:
                #print(gene)
                if gene[0] == gene[0].upper():
                    new_gene.append(gene)
        return list(sorted(new_gene))
        
        
    def interact(self, another_flower):
        my_gene = self.gene_series.split('-')
        her_gene = another_flower.get_gene_series().split('-')
        
        new_gene_list = []
        for ind, my_sub_gene in enumerate(my_gene):
            her_sub_gene = her_gene[ind]
            new_sub_genes = self.combine(my_sub_gene, her_sub_gene)
            new_gene_list.append(new_sub_genes)
        output_gene = []
        if len(my_gene) == 4:
            for r in new_gene_list[0]:
                for y in new_gene_list[1]:
                    for w in new_gene_list[2]:
                        for s in new_gene_list[3]:
                            new_flower = r+'-'+y+'-'+w+'-'+s
                            if new_flower in flower_dict[self.flower_name].keys():
                                output_gene.append(new_flower)
        else:    
            for r in new_gene_list[0]:
                for y in new_gene_list[1]:
                    for w in new_gene_list[2]:
                        new_flower = r+'-'+y+'-'+w
                        if new_flower in flower_dict[self.flower_name].keys():
                            output_gene.append(new_flower)
         
        return output_gene

In [6]:
class Flower(object):
    def __init__(self, flower, first_seed_gene, second_seed_gene, third_seed_gene):
        self.flower = flower
        self.first_seed_gene = first_seed_gene
        self.second_seed_gene = second_seed_gene
        self.third_seed_gene = third_seed_gene
        self.color_dict = flower_dict[flower]
        self.raw_breeding_table = None
        self.all_genes = None
    
    def get_current_permutations(self, all_current_breeds):
        '''get current breeding premutations'''
        breeding_pair = []
        for breed in all_current_breeds:
            breeding_pair.append([breed,breed])

        permutes = list(permutations(all_current_breeds,2))
        for pair in permutes:
            new_pair = sorted(pair)
            if new_pair not in breeding_pair:
                breeding_pair.append(new_pair)
        return sorted(breeding_pair)
    
    def get_color(self, genes):
        color_dict = flower_dict[self.flower]
        if genes in color_dict.keys():
            return color_dict[genes]
        else:
            return 'invalid'
        
    def get_all_genes(self):
        return self.all_genes
        
    def start_breeding(self):
        # basic seed breading
        current_breeds = [self.first_seed_gene, self.second_seed_gene, self.third_seed_gene]
        
        breeded_pair = []
        breeding_results_track = []
        
        keep_breading = True
        while keep_breading:
            # get possible breeding pairs:
            current_breeding_pair = self.get_current_permutations(current_breeds)
            
            # start breeding for each pair
            updated = 0
            for pair in current_breeding_pair:
                if pair in breeded_pair:
                    continue
                #print('Pair to breed: ', pair)
                flower1 = flowerGenetic(self.flower, pair[0])
                flower2 = flowerGenetic(self.flower, pair[1])
                new_breeds = flower1.interact(flower2)
                #print('Breeded results: ',new_breeds)
                breeding_results_track.append([pair, new_breeds])
                
                
                for new_breed in new_breeds:
                    if new_breed not in current_breeds:
                        #print('it is a new gene: ', new_breed)
                        current_breeds.append(new_breed)
                        updated+=1
            #print(updated)
            if updated ==0:
                keep_breading = False
            breeded_pair.append(pair)
            
        breeding_results = pd.DataFrame(breeding_results_track, columns = ['breading_pair', 'breading_results'])
        self.raw_breeding_table = breeding_results
        self.all_genes = current_breeds
        return breeding_results
    
    def get_breading_table(self):
        
        if self.raw_breeding_table is None:
            self.start_breeding()
        
        table = self.raw_breeding_table
        
        breeded = table.loc[table.breading_results.apply(lambda x:x != [])]
        breeded = breeded.copy()
        breeded['breading_pair_color'] = breeded['breading_pair'].apply(lambda x: list(map(lambda y: self.get_color(y), x)))
        breeded['breading_results_color'] = breeded['breading_results'].apply(lambda x: list(map(lambda y: self.get_color(y), x)))
        breeded['probability'] = breeded['breading_results'].apply(lambda x: list(map(lambda y: str(int(1.0/len(x)*100))+'%', x)))

        return breeded

In [7]:
lily = Flower('lily', 'RR-yy-Ss', 'rr-YY-ss','rr-yy-SS')
lily_results_table = lily.get_breading_table()
len(lily.get_all_genes()), lily_results_table.shape

(6, (17, 5))

In [9]:
lily_results_table.to_csv('lily_genes.csv',index=False)

In [10]:
mum = Flower('mum', 'RR-yy-WW', 'rr-YY-WW','rr-yy-Ww')
mum_results_table = mum.get_breading_table()
len(mum.get_all_genes()), mum_results_table.shape

(15, (138, 5))

In [11]:
mum_results_table.to_csv('mum_genes.csv',index=False)

In [12]:
rose = Flower('rose', 'RR-yy-WW-Ss', 'rr-YY-WW-ss','rr-yy-Ww-ss')
rose_results_table = rose.get_breading_table()
len(rose.get_all_genes()), rose_results_table.shape

(33, (504, 5))

In [18]:
rose_results_table.to_csv('rose_genes.csv',index=False)

In [13]:
cosmo = Flower('cosmo', 'RR-yy-ss', 'rr-YY-Ss','rr-yy-Ss')
cosmo_results_table = cosmo.get_breading_table()
len(cosmo.get_all_genes()), cosmo_results_table.shape

(11, (57, 5))

In [19]:
cosmo_results_table.to_csv('cosmo_genes.csv',index=False)

In [14]:
pansy = Flower('pansy', 'RR-yy-WW', 'rr-YY-WW','rr-yy-Ww')
pansy_results_table = pansy.get_breading_table()
len(pansy.get_all_genes()), pansy_results_table.shape

(15, (138, 5))

In [20]:
pansy_results_table.to_csv('pansy_genes.csv',index=False)

In [15]:
tulip = Flower('tulip', 'RR-yy-Ss', 'rr-YY-ss','rr-yy-Ss')
tulip_results_table = tulip.get_breading_table()
len(tulip.get_all_genes()), tulip_results_table.shape

(11, (59, 5))

In [21]:
tulip_results_table.to_csv('tulip_genes.csv',index=False)

In [16]:
hyacinth = Flower('hyacinth', 'RR-yy-Ww', 'rr-YY-WW','rr-yy-Ww')
hyacinth_results_table = hyacinth.get_breading_table()
len(hyacinth.get_all_genes()), hyacinth_results_table.shape

(15, (163, 5))

In [22]:
hyacinth_results_table.to_csv('hyacinth_genes.csv',index=False)

In [17]:
windflower = Flower('windflower', 'RR-oo-WW', 'rr-OO-WW','rr-oo-Ww')
windflower_results_table = windflower.get_breading_table()
len(windflower.get_all_genes()), windflower_results_table.shape

(15, (138, 5))

In [23]:
windflower_results_table.to_csv('windflower_genes.csv',index=False)

In [24]:
windflower_results_table

,breading_pair,breading_results,breading_pair_color,breading_results_color,probability
0,"[RR-oo-WW, RR-oo-WW]",[RR-oo-WW],"[Red (seed), Red (seed)]",[Red (seed)],[100%]
2,"[RR-oo-WW, rr-oo-Ww]","[Rr-oo-WW, Rr-oo-Ww]","[Red (seed), White (seed)]","[Red, Red]","[50%, 50%]"
3,"[rr-OO-WW, rr-OO-WW]",[rr-OO-WW],"[Orange (seed), Orange (seed)]",[Orange (seed)],[100%]
4,"[rr-OO-WW, rr-oo-Ww]","[rr-Oo-WW, rr-Oo-Ww]","[Orange (seed), White (seed)]","[Orange, Orange]","[50%, 50%]"
5,"[rr-oo-Ww, rr-oo-Ww]","[rr-oo-WW, rr-oo-Ww, rr-oo-ww]","[White (seed), White (seed)]","[White, White (seed), Blue]","[33%, 33%, 33%]"
...,...,...,...,...,...
163,"[rr-Oo-ww, rr-oo-ww]","[rr-Oo-ww, rr-oo-ww]","[Blue, Blue]","[Blue, Blue]","[50%, 50%]"
164,"[rr-oo-WW, rr-oo-WW]",[rr-oo-WW],"[White, White]",[White],[100%]
165,"[rr-oo-WW, rr-oo-Ww]","[rr-oo-WW, rr-oo-Ww]","[White, White (seed)]","[White, White (seed)]","[50%, 50%]"
166,"[rr-oo-WW, rr-oo-ww]",[rr-oo-Ww],"[White, Blue]",[White (seed)],[100%]
